In [ ]:
"""
## BELOW: TESTING Socrata NY Open Data API. 
####    Using request library to pull 311 data for Community Board

#### To DO 
1. Create Javascript that takes value of community board dropdown and passes it to query builder
2. Create Date reader. 

"""

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
from datetime import datetime
import openpyxl
import os
import csv
import requests 
import folium
import matplotlib
import numpy as np

In [3]:
# Geojson Endpoint for NYC Open Data - 311 2010 - Present
# https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

ENDPOINT = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson"
QUERY_SYMBOL = '?'
COMMUNITY_BOARD = 'community_board'
CB = "03 MANHATTAN"
base_url = ENDPOINT + QUERY_SYMBOL + COMMUNITY_BOARD + '=' + CB

In [4]:
# Create secondary query to test date function. 
# Documentation here: https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9

ENDPOINT = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson"
QUERY_SYMBOL = '?'
WHERE = "$where="
date = "2021-09-01T00:00:00.000"
date_operator = " > "
date_query = f"created_date{date_operator}'{date}'"
base_url = ENDPOINT + QUERY_SYMBOL + WHERE + date_query

In [5]:
# Current Time in floating timestamp data type, which is what Socrata uses. 
now = datetime.now()
dt_2 = now.strftime('%Y-%m-%dT%H:%M:%S.%fZ')[:-4]

In [6]:
print(dt_2)

2021-09-23T22:56:09.140


In [7]:
# See base url assembled from pieces above
base_url

"https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date > '2021-09-01T00:00:00.000'"

In [8]:
# Blank Space in url works with Socrata, but not geopandas, so replacing space with html hexadecimal space. 
base_url = base_url.replace(' ','%20')

In [9]:
cb3_complaints = requests.get(base_url)

In [10]:
# Get html response code
cb3_complaints

<Response [200]>

In [11]:
# Read in geojson data from socrata url created above 

cb3_complaints_geo = gpd.read_file(base_url)

In [12]:
# Make sure geodataframe was created
type(cb3_complaints_geo)

geopandas.geodataframe.GeoDataFrame

In [13]:
cb3_complaints_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
# Total bounds is returning null array. This is probably because there are null values in geometry column. 
# Follow Steps 1 and 2 to correct:

# 1 Create new geodataframe of rows with null geometry so that data is not lost
complaints_null_geo = cb3_complaints_geo[cb3_complaints_geo['geometry'].isna()]

# 2 Remove all rows with null geometry from original geodataframe 
cb3_complaints_geo = cb3_complaints_geo[cb3_complaints_geo['geometry'].notna()]

In [15]:
bounds = cb3_complaints_geo.total_bounds
a = np.mean(bounds[0:3:2]).round(3)
b = np.mean(bounds[1:4:2]).round(3)
data_centroid = [b,a]
print(data_centroid)

[40.71, -73.977]


In [17]:
cb3_complaints = cb3_complaints_geo.to_json()

In [18]:
type(cb3_complaints)

str

In [28]:
filepath_cb_complaints = r"/Users/calvindechicago/Desktop"

In [ ]:
#cb3_complaints_geo.to_file(filepath_cb_complaints, driver="GeoJSON")  

In [19]:
print(cb3_complaints)

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"address_type": null, "agency": "NYPD", "agency_name": "New York City Police Department", "bbl": "2033000001", "borough": "BRONX", "bridge_highway_direction": null, "bridge_highway_name": null, "bridge_highway_segment": null, "city": "BRONX", "closed_date": "2021-09-01T01:35:19", "community_board": "07 BRONX", "complaint_type": "Noise - Vehicle", "created_date": "2021-09-01T00:00:05", "cross_street_1": "EAST  194 STREET", "cross_street_2": "EAST  196 STREET", "descriptor": "Car/Truck Music", "due_date": null, "facility_type": null, "incident_address": "2645 BRIGGS AVENUE", "incident_zip": "10458", "intersection_street_1": "EAST  194 STREET", "intersection_street_2": "EAST  196 STREET", "landmark": "BRIGGS AVENUE", "latitude": "40.86535764249122", "location_address": "", "location_city": "", "location_state": "", "location_type": "Street/Sidewalk", "location_zip": "", "longitude": "-73.892495473103"

In [95]:



cb3_complaints_geo = cb3_complaints_geo[cb3_complaints_geo['geometry'].notna()]




In [96]:
cb3_complaints_geo.total_bounds

array([-74.23906546,  40.50809206, -73.71438452,  40.91102202])

In [113]:
# Get sorted unique community board values in data pull 

cbs = cb3_complaints_geo.loc[:,'community_board'].sort_values().unique()

In [ ]:
cbs

In [ ]:
# Create drop down selection html for each community board 

for i in cbs:
    i=f"""<option value="{i}">{i}</option>"""
    print(i)

In [127]:
mapcomplaints = folium.Map(location=data_centroid, tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

In [128]:
folium.features.GeoJson(cb3_complaints_geo,                                                                          
                       ).add_to(mapcomplaints)

In [129]:
# Creates Folium map
mapcomplaints

In [ ]:
# Create a Map instance
m = folium.Map(location=data_centroid, tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

#Plot a choropleth map
#Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=choropleth_data,
    name='Percentage of Cyclists',
    data=choropleth_data,
    columns=['geoid', 'pct_bike'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    #threshold_scale=[1, 2, 3, 4, 5],
    legend_name= 'Percentage of workers that bike to work').add_to(m)